In [1]:
%pip list | grep catboost
%pip install catboost --upgrade


Note: you may need to restart the kernel to use updated packages.


'grep' is not recognized as an internal or external command,
operable program or batch file.


Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.



In [ ]:
import pandas as pd
from catboost import CatBoostClassifier, Pool, cv
from sklearn.model_selection import train_test_split

# Load the data
data_path = "../data/[FINAL] National Pokedex Gen 9.xlsx - Pokedex.csv"
pokemon_df = pd.read_csv(data_path)

# Preprocess the data
# Drop unnecessary columns
columns_to_drop = ['National\nDex', 'Type I', 'Type II', 'Pokemon\nName', 'Pokedex Desc']
X = pokemon_df.drop(columns=[col for col in columns_to_drop if col in pokemon_df.columns])
y = pokemon_df[['Type I', 'Type II']]

# Handle missing values
X.fillna('Unknown', inplace=True)
y.fillna('Unknown', inplace=True)

# Concatenate target columns into a single column
y['Combined'] = y.apply(lambda row: '_'.join(row.values.astype(str)), axis=1)
y_combined = y['Combined']

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y_combined, test_size=0.2, random_state=42)

# Print the column names to check which columns are available
print("Columns in dataframe:", X.columns)

# Define categorical features (update this list based on the actual columns in the dataframe)
categorical_cols = ['Ability I', 'Ability II', 'Hidden Ability', 'Pokedex Color', 'Egg Group I', 'Egg Group II']

# Initialize CatBoostClassifier
model = CatBoostClassifier(iterations=1000, depth=6, learning_rate=0.1, loss_function='MultiClass', eval_metric='MultiClass')

# Perform k-fold cross-validation
cv_params = model.get_params()
cv_data = cv(
    Pool(X, y_combined, cat_features=categorical_cols),
    cv_params,
    fold_count=10,
    plot=True
)

# Train the model
model.fit(train_pool, eval_set=test_pool, verbose=100)

# Predict probabilities
predictions = model.predict_proba(test_pool)
print(predictions)

Columns in dataframe: Index(['HP', 'Attack', 'Defense', 'Special\nAttack', 'Special\nDefense',
       'Speed', 'Total', 'Ability I', 'Ability II', 'Hidden Ability',
       'Weight (kg)', 'Height (m)', 'Pokedex Color', 'Egg Group I',
       'Egg Group II'],
      dtype='object')


C:\Users\maxfe\AppData\Local\Temp\ipykernel_89328\4019455712.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y.fillna('Unknown', inplace=True)


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Training on fold [0/10]


CatBoostError: catboost/libs/train_lib/cross_validation.cpp:629: Cannot do stratified split: Target data is multi-dimensional